In [2]:
import graphlab as gl
from graphlab import SArray
import pandas as pd
import numpy as np

A newer version of GraphLab Create (v1.8) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [3]:
sample = gl.SFrame('../data/Homesite Quote Conversion/sample_submission.csv')
train=gl.SFrame('../data/Homesite Quote Conversion/train.csv')
test=gl.SFrame('../data/Homesite Quote Conversion/test.csv')

[INFO] 1453186891 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/howard/anaconda/envs/dato-env/lib/python2.7/site-packages/certifi/cacert.pem
1453186891 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to studiocardo@gmail.com and will expire on November 14, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-3929 - Server binary: /Users/howard/anaconda/envs/dato-env/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1453186891.log
[INFO] GraphLab Server Version: 1.7.1


PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.200091 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 173836 lines in 0.121303 secs.
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 2.66617 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,int,str,int,float,float,st

In [74]:
train.num_cols()

299

In [75]:
# What percentage of the quotes are converted 
#

float(train[train['QuoteConversion_Flag'] == 1].num_rows()) / float(train.num_rows())

0.1875107860695754

In [76]:
train.num_rows()

260753

In [77]:
train.num_cols()

299

In [78]:
test.num_cols()
test.column_names()

['QuoteNumber',
 'Original_Quote_Date',
 'Field6',
 'Field7',
 'Field8',
 'Field9',
 'Field10',
 'Field11',
 'Field12',
 'CoverageField1A',
 'CoverageField1B',
 'CoverageField2A',
 'CoverageField2B',
 'CoverageField3A',
 'CoverageField3B',
 'CoverageField4A',
 'CoverageField4B',
 'CoverageField5A',
 'CoverageField5B',
 'CoverageField6A',
 'CoverageField6B',
 'CoverageField8',
 'CoverageField9',
 'CoverageField11A',
 'CoverageField11B',
 'SalesField1A',
 'SalesField1B',
 'SalesField2A',
 'SalesField2B',
 'SalesField3',
 'SalesField4',
 'SalesField5',
 'SalesField6',
 'SalesField7',
 'SalesField8',
 'SalesField9',
 'SalesField10',
 'SalesField11',
 'SalesField12',
 'SalesField13',
 'SalesField14',
 'SalesField15',
 'PersonalField1',
 'PersonalField2',
 'PersonalField4A',
 'PersonalField4B',
 'PersonalField5',
 'PersonalField6',
 'PersonalField7',
 'PersonalField8',
 'PersonalField9',
 'PersonalField10A',
 'PersonalField10B',
 'PersonalField11',
 'PersonalField12',
 'PersonalField13',
 'P

In [79]:
train.column_names()

['QuoteNumber',
 'Original_Quote_Date',
 'QuoteConversion_Flag',
 'Field6',
 'Field7',
 'Field8',
 'Field9',
 'Field10',
 'Field11',
 'Field12',
 'CoverageField1A',
 'CoverageField1B',
 'CoverageField2A',
 'CoverageField2B',
 'CoverageField3A',
 'CoverageField3B',
 'CoverageField4A',
 'CoverageField4B',
 'CoverageField5A',
 'CoverageField5B',
 'CoverageField6A',
 'CoverageField6B',
 'CoverageField8',
 'CoverageField9',
 'CoverageField11A',
 'CoverageField11B',
 'SalesField1A',
 'SalesField1B',
 'SalesField2A',
 'SalesField2B',
 'SalesField3',
 'SalesField4',
 'SalesField5',
 'SalesField6',
 'SalesField7',
 'SalesField8',
 'SalesField9',
 'SalesField10',
 'SalesField11',
 'SalesField12',
 'SalesField13',
 'SalesField14',
 'SalesField15',
 'PersonalField1',
 'PersonalField2',
 'PersonalField4A',
 'PersonalField4B',
 'PersonalField5',
 'PersonalField6',
 'PersonalField7',
 'PersonalField8',
 'PersonalField9',
 'PersonalField10A',
 'PersonalField10B',
 'PersonalField11',
 'PersonalField12'

## 1. Data Scrubbing

### 1.1 Assign all NaN/None to -1

In [4]:
# Make sure you assign it back to the SFrame
#
cols = train.column_names()
for col in cols:
    train = train.fillna(col, -1)

## 1.2 Prepare base models

1. without the first column, QuoteNumber
2. (1). minus the Original_Quote_Date, but the Date are reformatted to have sep columns for [weekday, month, year]

### 1.2.1 Identify columns that are not conducive to training

In [98]:
dd=test['Original_Quote_Date'].head()
ddt=dd.str_to_datetime()
ddt

dtype: datetime
Rows: 10
[datetime.datetime(2014, 8, 12, 0, 0), datetime.datetime(2013, 9, 7, 0, 0), datetime.datetime(2013, 3, 29, 0, 0), datetime.datetime(2015, 3, 21, 0, 0), datetime.datetime(2014, 12, 10, 0, 0), datetime.datetime(2013, 7, 19, 0, 0), datetime.datetime(2014, 7, 28, 0, 0), datetime.datetime(2015, 1, 20, 0, 0), datetime.datetime(2014, 5, 28, 0, 0), datetime.datetime(2013, 7, 11, 0, 0)]

In [99]:
temp=ddt.split_datetime(column_name_prefix=None,limit=['weekday', 'month','year'])
temp

weekday,month,year
1,8,2014
5,9,2013
4,3,2013
5,3,2015
2,12,2014
4,7,2013
0,7,2014
1,1,2015
2,5,2014
3,7,2013


In [6]:
# Change Original_Quote_Date column to 3 sep of [year, month, weekday] -> give patterns, otherwise dates are useless
#
train['date'] = train['Original_Quote_Date'].str_to_datetime()

temp=train['date'].split_datetime(column_name_prefix=None,limit=['weekday', 'month','year'])
train['year']=temp['year']
train['month']=temp['month']
train['weekday']=temp['weekday']

# Do the same for the test data
#
test['date'] = test['Original_Quote_Date'].str_to_datetime()

temp=test['date'].split_datetime(column_name_prefix=None,limit=['weekday', 'month','year'])
test['year']=temp['year']
test['month']=temp['month']
test['weekday']=temp['weekday']

In [101]:
train.column_names()

['QuoteNumber',
 'Original_Quote_Date',
 'QuoteConversion_Flag',
 'Field6',
 'Field7',
 'Field8',
 'Field9',
 'Field10',
 'Field11',
 'Field12',
 'CoverageField1A',
 'CoverageField1B',
 'CoverageField2A',
 'CoverageField2B',
 'CoverageField3A',
 'CoverageField3B',
 'CoverageField4A',
 'CoverageField4B',
 'CoverageField5A',
 'CoverageField5B',
 'CoverageField6A',
 'CoverageField6B',
 'CoverageField8',
 'CoverageField9',
 'CoverageField11A',
 'CoverageField11B',
 'SalesField1A',
 'SalesField1B',
 'SalesField2A',
 'SalesField2B',
 'SalesField3',
 'SalesField4',
 'SalesField5',
 'SalesField6',
 'SalesField7',
 'SalesField8',
 'SalesField9',
 'SalesField10',
 'SalesField11',
 'SalesField12',
 'SalesField13',
 'SalesField14',
 'SalesField15',
 'PersonalField1',
 'PersonalField2',
 'PersonalField4A',
 'PersonalField4B',
 'PersonalField5',
 'PersonalField6',
 'PersonalField7',
 'PersonalField8',
 'PersonalField9',
 'PersonalField10A',
 'PersonalField10B',
 'PersonalField11',
 'PersonalField12'

### 1.2.2 Find columns that contain only constant

In [89]:
len(train['month'].unique())

1

In [7]:
cols = train.column_names()
constants = []

for col in cols:
    # x.count(x[0]) == len(x)
    
    if (len(train[col].unique()) == 1):
        constants.append(col)
        
# train_int.num_cols() #265
constants

['PropertyField6', 'GeographicField10A']

In [8]:
cols = test.column_names()
constants = []

for col in cols:
    # x.count(x[0]) == len(x)
    
    if (len(test[col].unique()) == 1):
        constants.append(col)
        
# train_int.num_cols() #265
constants

['PropertyField6', 'GeographicField10A']

### 1.2.3 Remove columns that we don't want

In [9]:
# Clean up
#
# each row in QuoteNumber is a unique identifier that provides no useful value in pattern building
#
train.remove_column('QuoteNumber')
test.remove_column('QuoteNumber')

#
# each row in QuoteNumber is a unique identifier that provides no useful value in pattern building
#
train.remove_columns(['date', 'Original_Quote_Date'])
test.remove_columns(['date', 'Original_Quote_Date'])

#
# Remove columns that only contain constant values
train.remove_columns(constants)
test.remove_columns(constants)

Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,CoverageField2A
E,16,0.9364,0.0006,"1,487",1.3045,N,4,4,4
F,11,0.9919,0.0038,564,1.1886,N,8,14,8
F,15,0.8945,0.0038,564,1.067,N,11,18,11
K,21,0.887,0.0004,"1,113",1.2665,Y,14,22,15
B,25,0.9153,0.0007,935,1.02,N,4,5,4
B,24,0.9403,0.0006,965,1.02,N,11,18,11
E,23,0.9392,0.0006,"1,487",1.3045,N,5,6,5
B,25,0.9153,0.0007,935,1.02,N,6,9,6
J,23,0.8928,0.0004,"1,113",1.2665,N,12,20,12
J,23,0.9691,0.0004,"1,165",1.2665,N,3,3,3


In [18]:
# Split training into train and validation
train, validation = train.random_split(0.9, seed=1)

## 2 Go straight to boosted tree w/ this train dataset

#### Let GL Create decide the classification..., we'll start w/ the full set

#### 2.1 xbg1.  Trained with full features. 

In [14]:
xbg_all = gl.boosted_trees_classifier.create(train, 
                                          target='QuoteConversion_Flag',
                                          max_iterations = 1000, 
                                          verbose = True,
                                          validation_set=None)

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 260753
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 297
PROGRESS: Number of unpacked features : 297
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter    Accuracy Elapsed time
PROGRESS:      0   9.071e-01       18.13s
PROGRESS:      1   9.071e-01       41.33s
PROGRESS:      2   9.111e-01       75.06s
PROGRESS:      3   9.111e-01      120.09s
PROGRESS:      4   9.128e-01      160.49s
PROGRESS:      5   9.146e-01      200.52s
PROGRESS:      6   9.149e-01      240.03s
PROGRESS:      7   9.165e-01      269.40s
PROGRESS:      8   9.163e-01      295.52s
PROGRESS:      9   9.177e-01      321.34s
PROGRESS:     10   9.200e-01      348.62s
PROGRESS:     11   9.199e-01      382.17s
PROGRESS:     12   9.208e-01      413.34s
PROGRESS:     13   9.209e-01    

## Run xbg_all against the validation set

In [20]:
xbg_all_testresults = xbg_all.predict(test)

In [21]:
submit = gl.SFrame('../data/Homesite Quote Conversion/sample_submission.csv')
submit.remove_column('QuoteConversion_Flag')
submit.add_column(xbg_all_testresults, name='QuoteConversion_Flag')
submit.export_csv('../data/Homesite Quote Conversion/xbg_all_1000.csv')

PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.108726 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 173836 lines in 0.09 secs.


In [ ]:
top_features = xbg_all.get_feature_importance()
top_features

In [11]:
features2 = top_features['name'].unique()
features2 = features2[:14]

In [12]:
#model_2 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features2)
#
# Use the top 14 features from above
# use the training set w/ separate [y,m,wkday] columns and no quote_num
#
# Delete the two constant columns
#

xbg_2 = gl.boosted_trees_classifier.create(train, 
                                           target='QuoteConversion_Flag', 
                                           features=features2, 
                                           max_iterations=5000,
                                           max_depth = 10,
                                           random_seed = 37)


PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 247769
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 14
PROGRESS: Number of unpacked features : 14
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.127e-01   8.126e-01        1.19s
PROGRESS:      1   8.128e-01   8.127e-01        2.40s
PROGRESS:      2   8.128e-01   8.126e-01        3.59s
PROGRESS:      3   8.127e-01   8.125e-01        4.79s
PROGRESS:      4   8.127e-01   8.125e-01        5.98s
PROGRESS:      5   8.128e-01   8.126e-01        7.19s
PROGRESS:      6   8.128e-01   8

In [13]:
xbg_2_results = xbg_2.predict(test)

In [14]:
submit = gl.SFrame('../data/Homesite Quote Conversion/sample_submission.csv')
submit.remove_column('QuoteConversion_Flag')
submit.add_column(xbg_2_results, name='QuoteConversion_Flag')
submit.export_csv('../output/Homesite Quote Conversion/xbg_2_3000.csv')

PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.123085 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 173836 lines in 0.101154 secs.


### Peer features 1.

In [11]:
# https://www.kaggle.com/mpearmain/homesite-quote-conversion/xgboost-benchmark/log

features_p1 = ['Field6', 'Field10', 'Field12', 'CoverageField8', 'CoverageField9', 'SalesField7', 'PersonalField7', 'PersonalField16',
'PersonalField17',
'PersonalField18',
'PersonalField19', 
'PropertyField3', 
'PropertyField4', 
'PropertyField5', 
'PropertyField7', 
'PropertyField14', 
'PropertyField28', 
'PropertyField30', 
'PropertyField31', 
'PropertyField32', 
'PropertyField33', 
'PropertyField34', 
'PropertyField36', 
'PropertyField37', 
'PropertyField38', 
'GeographicField63', 
'GeographicField64']

xbg_p1 = gl.boosted_trees_classifier.create(train, 
                                            target='QuoteConversion_Flag', 
                                            features=features_p1, 
                                            max_iterations=5000,
                                            max_depth = 10,
                                            step_size = 0.025,
                                            row_subsample = 0.8,
                                            column_subsample = 0.8,
                                            random_seed = 37)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 247644
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 27
PROGRESS: Number of unpacked features : 27
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.243e-01   8.215e-01        2.00s
PROGRESS:      1   8.247e-01   8.220e-01        4.00s
PROGRESS:      2   8.266e-01   8.225e-01        5.89s
PROGRESS:      3   8.270e-01   8.223e-01        7.94s
PROGRESS:      4   8.268e-01   8.210e-01        9.92s
PROGRESS:      5   8.270e-01   8.214e-01       11.92s
PROGRESS:      6   8.271e-01   8

In [18]:
xbg_p1.summary()

Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 247617
Number of classes             : 2
Number of feature columns     : 27
Number of unpacked features   : 27

Settings
--------
Number of trees               : 3000
Max tree depth                : 6
Train accuracy                : 0.9013
Validation accuracy           : 0.8158
Training time (sec)           : 4852.0952



In [68]:
# Test the xbg_p1 model w/ test data
#
xbg_p1_results = xbg_p1.predict(test)


In [69]:

# Reformat the test results to conform the submission format
#
submit = gl.SFrame('../data/Homesite Quote Conversion/sample_submission.csv')
submit.remove_column('QuoteConversion_Flag')
submit.add_column(xbg_p1_results, name='QuoteConversion_Flag')
submit.export_csv('../output/Homesite Quote Conversion/xbg_p1_3000.csv')

PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.107885 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/howard/Desktop/SH3.0/Kaggle/Homesite_QuoteConversion/sample_submission.csv
PROGRESS: Parsing completed. Parsed 173836 lines in 0.087709 secs.


## Model 3

In [ ]:
features3 = top_features['name'][:10]
features3
#model_3 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features3)
model_3 = gl.boosted_trees_classifier.create(train, target='QuoteConversion_Flag', features=features3, max_iterations=200)

In [ ]:
features4 = top_features['name'][:7]
features4
model_4 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features4)

In [ ]:
features5 = ['PersonalField9', 'PropertyField37', 'SalesField5', 'PropertyField29', 'SalesField1A', 'PersonalField1']
model_5 = gl.classifier.create(train, target='QuoteConversion_Flag', features=features5)